# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

data = pd.read_csv('/datasets/Churn.csv',  sep=',')

# Предобработка
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1) #удалим ненужные столбцы
data = pd.get_dummies(data, drop_first=True) # получение дамми-признаков
data['Tenure'][data['Tenure'].isnull()]=0 # заменим пропуски 
data['Tenure']=data['Tenure'].astype(int)

# Деление данных
data_train, data_valid = train_test_split(data, test_size=0.40, random_state=12345)
data_test, data_valid = train_test_split(data_valid, test_size=0.50, random_state=12345)
features_train = data_train.drop(['Exited'], axis=1)
target_train = data_train['Exited']
features_valid = data_valid.drop(['Exited'], axis=1)
target_valid = data_valid['Exited']
features_test = data_test.drop(['Exited'], axis=1)
target_test = data_test['Exited']
# Маштабирование
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance','NumOfProducts','EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_train.head()



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7479,-0.886751,-0.373192,1.104696,1.232271,-0.891560,1,0,-0.187705,0,1,1
3411,0.608663,-0.183385,1.104696,0.600563,-0.891560,0,0,-0.333945,0,0,0
6027,2.052152,0.480939,-0.503694,1.027098,0.830152,0,1,1.503095,1,0,1
1247,-1.457915,-1.417129,0.461340,-1.233163,0.830152,1,0,-1.071061,0,0,1
3716,0.130961,-1.132419,-0.825373,1.140475,-0.891560,0,0,1.524268,1,0,0


#### Вывод
 - В таблице присутствуют бесполезные для модели столбцы (индекс строки в данных, уникальный идентификатор клиента, фамилия) удалим их.
 - Разделим категориальные признаки (Страна,Пол) с помощью One-hot-encoding.
 - В столбце - 'количество недвижимости', отсутствует часть данных. Заполним пропуски. 
 - разделим данные на три выборки.
 - отмасштабируем столбцы с небинарными данными.


# 2. Исследование задачи

In [3]:
# Исследование баланса классов
class_frequency = data['Exited'].value_counts(normalize=True)
print('Баланс классов')
print(class_frequency)
class_frequency.plot(kind='bar')

# Обучение модели без учёта дисбаланса
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Значение F1-меры',f1_score(target_valid,predicted_valid))

Баланс классов
0    0.7963
1    0.2037
Name: Exited, dtype: float64
Значение F1-меры 0.2743055555555555


#### Исследование баланса классов
 - Наблюдается дисбаланс положительного класса в целевом признаке 'Exited'. отрицательных признаков в пять раз больше.
 
#### Обучение модели без учёта дисбаланса
 - применим метод логистической регрессии для проверки модели обученной на несбалансированных данных.
 - Значение F1-меры - 0.27. Слишком низко. Необходима борьба с дисбалансом.


# 3. Борьба с дисбалансом

In [7]:
# Взвешивание классов
model2 = LogisticRegression(random_state=12345, solver='liblinear',class_weight='balanced')
model2.fit(features_train, target_train)
predicted_valid2 = model2.predict(features_valid)
predicted_train2 = model2.predict(features_train)
probabilities_one_valid2 = model2.predict_proba(features_valid)[:, 1]
print('Взвешивание классов')
print("Лог. регрессия F1 valid:",f1_score(target_valid,predicted_valid2))
print("Лог. регрессия train F1:", f1_score(target_train, predicted_train2))
print('Лог. регрессия valid auc_roc',roc_auc_score(target_valid, probabilities_one_valid2))


# Увеличение выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

print()
print('Увеличение выборки')
# Логистическая регрессия

model3 = LogisticRegression(random_state=12345, solver='liblinear')
model3.fit(features_upsampled, target_upsampled)
predicted_valid3 = model3.predict(features_valid)
predicted_train3 = model3.predict(features_train)
probabilities_one_valid2 = model3.predict_proba(features_valid)[:, 1]
print("Лог. регрессия valid F1:", f1_score(target_valid, predicted_valid3))
print("Лог. регрессия train F1:", f1_score(target_train, predicted_train3))
print('Лог. регрессия valid auc_roc',roc_auc_score(target_valid, probabilities_one_valid2))




# Уменьшение выборки
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

print()
print('Уменьшение выборки')
# Логистическая регрессия
model3 = LogisticRegression(random_state=12345,solver='liblinear')
model3.fit(features_downsampled, target_downsampled)
predicted_valid3 = model3.predict(features_valid)
predicted_train3 = model3.predict(features_train)
probabilities_one_valid2 = model3.predict_proba(features_valid)[:, 1]
print("Лог. регрессия valid F1:", f1_score(target_valid, predicted_valid3))
print("Лог. регрессия train F1:", f1_score(target_train, predicted_train3))
print('Лог. регрессия valid auc_roc',roc_auc_score(target_valid, probabilities_one_valid2))


Взвешивание классов
Лог. регрессия F1 valid: 0.4797238999137188
Лог. регрессия train F1: 0.5
Лог. регрессия valid auc_roc 0.7417876058170719

Увеличение выборки
Лог. регрессия valid F1: 0.486610558530987
Лог. регрессия train F1: 0.4796238244514106
Лог. регрессия valid auc_roc 0.7420004767108749

Уменьшение выборки
Лог. регрессия valid F1: 0.4305239179954442
Лог. регрессия train F1: 0.42554858934169276
Лог. регрессия valid auc_roc 0.7358631989698248


#### Вывод
В борьбе с дисбалансом увеличение выборки показало лучший результат по всем параметрам, чем взвешивание классов по регрессионной модели. Попробуем увеличение выборки для моделей с деревьями.

In [5]:
# Классификация деревом решений, Увеличенная выбока:
F1=0
F11=0
max_depth1=1
for estim in range(1, 30):
    model4 = DecisionTreeClassifier(random_state=12345, max_depth=estim)
    model4.fit(features_upsampled, target_upsampled)
    predicted_valid4 = model4.predict(features_valid)
    predicted_train4 = model4.predict(features_train)
    if F1 <= f1_score(target_valid, predicted_valid4):
            F1 = f1_score(target_valid, predicted_valid4)
            F11 = f1_score(target_train, predicted_train4)
            max_depth1 = estim
            probabilities_one_valid4 = model4.predict_proba(features_valid)[:, 1]
print("Max F1 Дерево решений")
print("валидационная выборка: {:.2f}".format(F1),end=" ; ")
print('auc_roc: {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid4)),end=" ; ")
print("тренировочная выборка: {:.2f}".format(F11),end=" ; ")
print("max_depth = ", max_depth1)

# Классификация случайным лесом, Увеличенная выбока:
valid=0
train=1
max_depth=1
n_estimators=1
for i in range(1,20):
    for estim in range(1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=i, max_depth=estim)# создать модель
        model.fit(features_upsampled, target_upsampled)# обучить модель
        predicted_valid = model.predict(features_valid)
        predicted_train = model.predict(features_train)
        if valid < f1_score(predicted_valid, target_valid) or train > f1_score(predicted_train, target_train):
            valid = f1_score(predicted_valid, target_valid)
            train = f1_score(predicted_train, target_train)
            model5=model
            max_depth = estim
            n_estimators = i
            probabilities_one_valid5 = model5.predict_proba(features_valid)[:, 1]

print('Max F1 Случайный лес')        
print("валидационная выборка:{:.2f}".format(valid),end=" ; ")
print('auc_roc: {:.2f}'.format(roc_auc_score(target_valid, probabilities_one_valid5)),end=" ; ")
print("тренировочная выборка:{:.2f}".format(train),end=" ; ")
print("max_depth = ", max_depth,end=" ; ")
print("n_estimators = ", n_estimators)


Max F1 Дерево решений
валидационная выборка: 0.55 ; auc_roc: 0.83 ; тренировочная выборка: 0.54 ; max_depth =  5
Max F1 Случайный лес
валидационная выборка:0.61 ; auc_roc: 0.84 ; тренировочная выборка:0.71 ; max_depth =  9 ; n_estimators =  19


#### Вывод
 - Для Увеличенной выборки модель случайного леса показала намного выше F1=0,61 чем у дерева решений, так же показатель auc_roc 0.84 свидетельствует, что модель является адекватной.

# 4. Тестирование модели

In [6]:
predicted_test = model5.predict(features_test)
probabilities_one_test = model5.predict_proba(features_test)[:, 1]
print("Тестовая выборка, Случайный лес F1:{:.2f}".format(f1_score(target_test, predicted_test)),end=" ; ")
print('auc_roc:{:.2f}'.format(roc_auc_score(target_test, probabilities_one_test)))


Тестовая выборка, Случайный лес F1:0.60 ; auc_roc:0.85


#### Вывод
- Тестирование лучшей модели случайного леса дало приемлемые результаты: F1 мера -0,6.  auc_roc для тестовой выборки -0.85. Модель адекватная.